# 第6章: Regression Models with Multiple Regressors（重回帰モデル）練習問題

この章では重回帰モデル、省略変数バイアス、多重共線性について学びます。

In [ ]:
# 必要なパッケージの読み込み
library(AER)
data("CASchools")
CASchools$STR <- CASchools$students / CASchools$teachers
CASchools$score <- (CASchools$read + CASchools$math) / 2

---
## 問題1: 重回帰モデルの推定

テストスコア(score)を被説明変数、学生教師比率(STR)と英語学習者の割合(english)を説明変数とする重回帰モデルを推定してください。

1. モデルを推定
2. 結果を表示し、各係数を解釈
3. 単回帰(score ~ STR)との係数の違いを確認

In [ ]:
# 問題1の解答欄



### 模範解答

In [ ]:
# 模範解答
# 単回帰モデル
model_simple <- lm(score ~ STR, data = CASchools)

# 1. 重回帰モデル
model_multi <- lm(score ~ STR + english, data = CASchools)

# 2. 結果の表示
cat("=== 重回帰モデルの結果 ===", "\n")
summary(model_multi)

# 3. 係数の比較
cat("\n=== 係数の比較 ===", "\n")
cat("単回帰モデルの STR 係数:", coef(model_simple)[2], "\n")
cat("重回帰モデルの STR 係数:", coef(model_multi)[2], "\n\n")

# 解釈
cat("=== 解釈 ===", "\n")
cat("STR の係数: 英語学習者の割合を一定として、\n")
cat("STRが1増加するとスコアは約", round(coef(model_multi)[2], 2), "点変化\n\n")
cat("english の係数: STRを一定として、\n")
cat("英語学習者の割合が1%増加するとスコアは約", round(coef(model_multi)[3], 2), "点変化\n")

---
## 問題2: 省略変数バイアス（OVB）

省略変数バイアスの方向を理論的に予測し、実際のデータで確認します。

1. STRとenglishの相関を計算
2. englishとscoreの相関（STRを制御しない場合）を計算
3. OVBの公式を使って、englishを省略した場合のバイアスの方向を予測
4. 実際の係数の差と比較

In [ ]:
# 問題2の解答欄



### 模範解答

In [ ]:
# 模範解答
# 1. STRとenglishの相関
cor_STR_english <- cor(CASchools$STR, CASchools$english)
cat("STRとenglishの相関:", cor_STR_english, "\n")

# 2. englishとscoreの相関
cor_english_score <- cor(CASchools$english, CASchools$score)
cat("englishとscoreの相関:", cor_english_score, "\n\n")

# 補助回帰: english ~ STR
aux_reg <- lm(english ~ STR, data = CASchools)
delta <- coef(aux_reg)[2]  # englishに対するSTRの効果

# 3. OVBの計算
# OVB = δ × β_english
# δ: englishに対するSTRの回帰係数
# β_english: 真のモデルでのenglishの係数

beta_english <- coef(model_multi)[3]  # 重回帰からのenglishの係数

OVB <- delta * beta_english
cat("=== OVBの計算 ===", "\n")
cat("補助回帰の係数 δ =", delta, "\n")
cat("englishの真の係数 β_english =", beta_english, "\n")
cat("推定されるOVB = δ × β_english =", OVB, "\n\n")

# 4. 実際の係数差との比較
actual_diff <- coef(model_simple)[2] - coef(model_multi)[2]
cat("実際の係数差（単回帰 - 重回帰）:", actual_diff, "\n\n")

cat("=== 解釈 ===", "\n")
cat("STRとenglishは正の相関（", round(cor_STR_english, 3), "）\n")
cat("englishはscoreに負の影響（", round(beta_english, 3), "）\n")
cat("→ englishを省略するとSTRの係数は負の方向にバイアス\n")

---
## 問題3: 自由度調整済み決定係数

単回帰モデルと重回帰モデルの決定係数を比較します。

1. 各モデルのR²を計算
2. 各モデルの自由度調整済みR²を計算
3. なぜ自由度調整済みR²を使うべきかを説明

In [ ]:
# 問題3の解答欄



### 模範解答

In [ ]:
# 模範解答
model_simple <- lm(score ~ STR, data = CASchools)
model_multi <- lm(score ~ STR + english, data = CASchools)

# 1-2. R²と調整済みR²
cat("=== 単回帰モデル ===", "\n")
cat("R² =", summary(model_simple)$r.squared, "\n")
cat("調整済みR² =", summary(model_simple)$adj.r.squared, "\n\n")

cat("=== 重回帰モデル ===", "\n")
cat("R² =", summary(model_multi)$r.squared, "\n")
cat("調整済みR² =", summary(model_multi)$adj.r.squared, "\n\n")

# 手計算での確認
n <- nrow(CASchools)
k_simple <- 1  # 説明変数の数
k_multi <- 2

R2_simple <- summary(model_simple)$r.squared
R2_multi <- summary(model_multi)$r.squared

adj_R2_simple <- 1 - (1 - R2_simple) * (n - 1) / (n - k_simple - 1)
adj_R2_multi <- 1 - (1 - R2_multi) * (n - 1) / (n - k_multi - 1)

cat("=== 手計算での確認 ===", "\n")
cat("単回帰の調整済みR²:", adj_R2_simple, "\n")
cat("重回帰の調整済みR²:", adj_R2_multi, "\n\n")

# 3. 説明
cat("=== なぜ調整済みR²を使うか ===", "\n")
cat("R²は説明変数を追加すると必ず増加する。\n")
cat("調整済みR²は、説明変数の追加による自由度の減少を考慮。\n")
cat("無関係な変数を追加した場合、調整済みR²は減少する。\n")
cat("→ モデル選択において、より適切な指標となる。\n")

---
## 問題4: 多重共線性の検出

説明変数間の相関が高い場合の問題を確認します。

1. STR, english, lunch（無料昼食を受ける生徒の割合）の相関行列を計算
2. VIF（分散拡大係数）を計算
3. 多重共線性が推定に与える影響を確認

In [ ]:
# 問題4の解答欄



### 模範解答

In [ ]:
# 模範解答
# install.packages("car")  # VIF計算用
library(car)

# 1. 相関行列
vars <- c("STR", "english", "lunch")
cor_matrix <- cor(CASchools[, vars])
cat("=== 相関行列 ===", "\n")
print(round(cor_matrix, 3))

# 重回帰モデル
model_full <- lm(score ~ STR + english + lunch, data = CASchools)

# 2. VIFの計算
cat("\n=== VIF（分散拡大係数）===", "\n")
print(vif(model_full))

# 3. 係数の比較
cat("\n=== 多重共線性の影響 ===", "\n")

# englishのみを含むモデル
model_eng <- lm(score ~ STR + english, data = CASchools)
# lunchのみを含むモデル
model_lunch <- lm(score ~ STR + lunch, data = CASchools)

cat("STRの係数の比較:\n")
cat("score ~ STR + english:", coef(model_eng)[2], "\n")
cat("score ~ STR + lunch:", coef(model_lunch)[2], "\n")
cat("score ~ STR + english + lunch:", coef(model_full)[2], "\n\n")

cat("englishとlunchの相関が高い（", round(cor_matrix[2,3], 3), "）\n")
cat("→ 両方を含めると係数の標準誤差が大きくなる\n")

---
## 問題5: 完全多重共線性

完全多重共線性が発生するケースを確認します。

1. 完全多重共線性を持つ変数を作成（例: STR_double = 2 * STR）
2. この変数を含めて回帰分析を試み、何が起こるか確認
3. ダミー変数トラップの例を確認

In [ ]:
# 問題5の解答欄



### 模範解答

In [ ]:
# 模範解答
# 1-2. 完全多重共線性
CASchools$STR_double <- 2 * CASchools$STR

cat("=== 完全多重共線性の例 ===", "\n")
model_collinear <- lm(score ~ STR + STR_double, data = CASchools)
cat("係数:\n")
print(coef(model_collinear))
cat("\n→ Rは自動的に片方の変数をNAとして除外する\n\n")

# 3. ダミー変数トラップ
# カテゴリカル変数を作成
CASchools$size_small <- ifelse(CASchools$STR <= 18, 1, 0)
CASchools$size_medium <- ifelse(CASchools$STR > 18 & CASchools$STR <= 22, 1, 0)
CASchools$size_large <- ifelse(CASchools$STR > 22, 1, 0)

# 全てのダミー変数を含める（ダミー変数トラップ）
cat("=== ダミー変数トラップ ===", "\n")
model_dummy_trap <- lm(score ~ size_small + size_medium + size_large, data = CASchools)
cat("全ダミー変数を含めた場合:\n")
print(coef(model_dummy_trap))

# 正しい方法（1つを参照カテゴリとして除外）
cat("\n正しい方法（largeを参照カテゴリとして除外）:\n")
model_dummy_correct <- lm(score ~ size_small + size_medium, data = CASchools)
print(coef(model_dummy_correct))

cat("\n=== 解釈 ===", "\n")
cat("切片: 大規模クラス（参照カテゴリ）の平均スコア\n")
cat("size_small: 小規模クラスと大規模クラスのスコア差\n")
cat("size_medium: 中規模クラスと大規模クラスのスコア差\n")

---
## 問題6: OLS仮定の確認

重回帰モデル score ~ STR + english について、OLS仮定を確認します。

1. 残差の正規性を確認（Q-Qプロット）
2. 残差の均一分散性を確認
3. 残差と説明変数の関係を確認

In [ ]:
# 問題6の解答欄



### 模範解答

In [ ]:
# 模範解答
model <- lm(score ~ STR + english, data = CASchools)

par(mfrow = c(2, 2))

# 1. 正規性の確認
qqnorm(resid(model), main = "正規Q-Qプロット")
qqline(resid(model), col = "red")

# 2. 均一分散性の確認
plot(fitted(model), resid(model),
     main = "残差 vs. 予測値",
     xlab = "予測値", ylab = "残差",
     pch = 19, col = rgb(0, 0, 1, 0.5))
abline(h = 0, col = "red")

# 3. 残差と説明変数の関係
plot(CASchools$STR, resid(model),
     main = "残差 vs. STR",
     xlab = "STR", ylab = "残差",
     pch = 19, col = rgb(0, 0, 1, 0.5))
abline(h = 0, col = "red")

plot(CASchools$english, resid(model),
     main = "残差 vs. english",
     xlab = "english", ylab = "残差",
     pch = 19, col = rgb(0, 0, 1, 0.5))
abline(h = 0, col = "red")

par(mfrow = c(1, 1))

# 統計的検定
library(lmtest)
cat("\n=== Breusch-Pagan検定（均一分散）===", "\n")
print(bptest(model))

---
## 問題7: 標準化回帰係数

変数のスケールが異なる場合の係数の比較のために、標準化回帰係数を計算します。

1. 各変数を標準化（平均0、標準偏差1）してから回帰分析を実行
2. 標準化係数を解釈
3. どの変数の影響が最も大きいか判断

In [ ]:
# 問題7の解答欄



### 模範解答

In [ ]:
# 模範解答
# 変数の標準化
CASchools$score_std <- scale(CASchools$score)
CASchools$STR_std <- scale(CASchools$STR)
CASchools$english_std <- scale(CASchools$english)
CASchools$lunch_std <- scale(CASchools$lunch)

# 1. 標準化変数での回帰
model_std <- lm(score_std ~ STR_std + english_std + lunch_std, data = CASchools)

cat("=== 標準化回帰係数 ===", "\n")
print(summary(model_std)$coefficients)

# 元の係数との比較
model_orig <- lm(score ~ STR + english + lunch, data = CASchools)

cat("\n=== 元の回帰係数 ===", "\n")
print(summary(model_orig)$coefficients[, 1:2])

# 2-3. 解釈
cat("\n=== 解釈 ===", "\n")
std_coefs <- coef(model_std)[-1]  # 切片を除く
cat("標準化係数の絶対値:\n")
print(sort(abs(std_coefs), decreasing = TRUE))

cat("\n最も影響が大きい変数: lunch（無料昼食を受ける生徒の割合）\n")
cat("lunchが1標準偏差増加すると、scoreは約", 
    round(abs(std_coefs[3]), 3), "標準偏差変化\n")

---
## 問題8: 偏回帰プロット

各説明変数の部分的な効果を視覚化する偏回帰プロット（added-variable plot）を作成します。

1. score ~ STR + english モデルについて、STRの偏回帰プロットを作成
2. car パッケージの avPlots() 関数を使用して全ての偏回帰プロットを作成

In [ ]:
# 問題8の解答欄



### 模範解答

In [ ]:
# 模範解答
library(car)

model <- lm(score ~ STR + english, data = CASchools)

# 1. 手動でSTRの偏回帰プロット
# Step 1: scoreをenglishで回帰し、残差を取得
resid_score <- resid(lm(score ~ english, data = CASchools))

# Step 2: STRをenglishで回帰し、残差を取得
resid_STR <- resid(lm(STR ~ english, data = CASchools))

# Step 3: 残差同士の散布図
par(mfrow = c(1, 2))

plot(resid_STR, resid_score,
     main = "STRの偏回帰プロット（手動）",
     xlab = "STR | english",
     ylab = "score | english",
     pch = 19, col = rgb(0, 0, 1, 0.5))
abline(lm(resid_score ~ resid_STR), col = "red", lwd = 2)

# 傾きの確認
cat("偏回帰の傾き:", coef(lm(resid_score ~ resid_STR))[2], "\n")
cat("重回帰のSTR係数:", coef(model)[2], "\n\n")

# 2. avPlots()を使用
avPlots(model, main = "Added-Variable Plots")

par(mfrow = c(1, 1))

---
## 問題9: モデル比較

異なる説明変数の組み合わせでモデルを比較します。

1. 以下の5つのモデルを推定
   - Model 1: score ~ STR
   - Model 2: score ~ STR + english
   - Model 3: score ~ STR + lunch
   - Model 4: score ~ STR + english + lunch
   - Model 5: score ~ STR + english + lunch + calworks + income

2. 各モデルの調整済みR²を比較
3. AIC/BICを使ったモデル選択

In [ ]:
# 問題9の解答欄



### 模範解答

In [ ]:
# 模範解答
# 1. モデルの推定
model1 <- lm(score ~ STR, data = CASchools)
model2 <- lm(score ~ STR + english, data = CASchools)
model3 <- lm(score ~ STR + lunch, data = CASchools)
model4 <- lm(score ~ STR + english + lunch, data = CASchools)
model5 <- lm(score ~ STR + english + lunch + calworks + income, data = CASchools)

# 2. 調整済みR²の比較
models <- list(model1, model2, model3, model4, model5)
model_names <- c("Model 1", "Model 2", "Model 3", "Model 4", "Model 5")

comparison <- data.frame(
  Model = model_names,
  R2 = sapply(models, function(m) summary(m)$r.squared),
  Adj_R2 = sapply(models, function(m) summary(m)$adj.r.squared),
  AIC = sapply(models, AIC),
  BIC = sapply(models, BIC),
  Vars = c(1, 2, 2, 3, 5)
)

cat("=== モデル比較 ===", "\n")
print(round(comparison, 3))

# 3. 最良モデルの選択
cat("\n=== モデル選択 ===", "\n")
cat("調整済みR²が最大: Model", which.max(comparison$Adj_R2), "\n")
cat("AICが最小: Model", which.min(comparison$AIC), "\n")
cat("BICが最小: Model", which.min(comparison$BIC), "\n")

---
## 問題10: シミュレーション - OLSの一致性

サンプルサイズを増やすとOLS推定量が真の値に収束することを確認します。

真のモデル: Y = 5 + 2X₁ + 3X₂ + u

1. サンプルサイズ n = 50, 100, 500, 1000, 5000 でシミュレーション
2. 各サンプルサイズで1000回推定を繰り返し
3. 推定量の分布がサンプルサイズとともに真の値に収束することを確認

In [ ]:
# 問題10の解答欄



### 模範解答

In [ ]:
# 模範解答
set.seed(123)

# 真のパラメータ
beta0_true <- 5
beta1_true <- 2
beta2_true <- 3
sigma <- 2

sample_sizes <- c(50, 100, 500, 1000, 5000)
n_sim <- 1000

results <- data.frame()

for (n in sample_sizes) {
  beta1_estimates <- numeric(n_sim)
  beta2_estimates <- numeric(n_sim)
  
  for (i in 1:n_sim) {
    X1 <- rnorm(n, 0, 1)
    X2 <- rnorm(n, 0, 1)
    u <- rnorm(n, 0, sigma)
    Y <- beta0_true + beta1_true * X1 + beta2_true * X2 + u
    
    model <- lm(Y ~ X1 + X2)
    beta1_estimates[i] <- coef(model)[2]
    beta2_estimates[i] <- coef(model)[3]
  }
  
  results <- rbind(results, data.frame(
    n = n,
    beta1_mean = mean(beta1_estimates),
    beta1_sd = sd(beta1_estimates),
    beta2_mean = mean(beta2_estimates),
    beta2_sd = sd(beta2_estimates)
  ))
}

cat("=== シミュレーション結果 ===", "\n")
cat("真の値: β₁ = 2, β₂ = 3\n\n")
print(round(results, 4))

# 視覚化
par(mfrow = c(1, 2))

plot(results$n, results$beta1_sd, type = "b",
     main = "β̂₁の標準偏差 vs. サンプルサイズ",
     xlab = "サンプルサイズ", ylab = "標準偏差",
     log = "x", pch = 19, col = "blue")

plot(results$n, results$beta2_sd, type = "b",
     main = "β̂₂の標準偏差 vs. サンプルサイズ",
     xlab = "サンプルサイズ", ylab = "標準偏差",
     log = "x", pch = 19, col = "red")

par(mfrow = c(1, 1))

cat("\n→ サンプルサイズが増加するにつれて、推定量の分散は減少し、\n")
cat("  推定量は真の値に収束する（一致性）\n")